In [ ]:
import os
import pandas as pd
import geopandas as gpd

In [ ]:
column_name_mapper = {
  'obs_value': 'value',
  'c_age_name': 'age',
  'age_name': 'age',
  'gender_name': 'gender'
}
age_value_mapper = {
  'All categories: Age 16+': 'Aged 16+',
  'Aged 16 to 24': 'Aged 16-24'
}

In [ ]:
claimant_columns = ['GEOGRAPHY_CODE', 'GEOGRAPHY_NAME', 'GENDER_NAME', 'AGE_NAME', 'MEASURE_NAME', 'OBS_VALUE']
claimants = pd.read_csv('../data/raw/claimants-by-pcon-2010-latest.csv', usecols=claimant_columns)
claimants.columns = claimants.columns.str.lower()
claimants = claimants.rename(columns=column_name_mapper)
claimants = claimants.replace({ 'age': age_value_mapper })
claimants

This is population estimates for England and Wales only!

In [ ]:
population_columns = ['GEOGRAPHY_CODE', 'GEOGRAPHY_NAME', 'GENDER_NAME', 'C_AGE_NAME', 'OBS_VALUE']
population = pd.read_csv('../data/raw/population-estimates-by-pcon-2010-latest.csv', usecols=population_columns)
population.columns = population.columns.str.lower()
population['measure_name'] = 'Population estimate'
population = population.rename(columns=column_name_mapper)
population = population.replace({ 'age': age_value_mapper })
population

In [ ]:
data = pd.concat([population, claimants]).pivot(index=['geography_code', 'geography_name', 'age', 'gender'], columns='measure_name', values='value')
data

In [ ]:
data['Claimants percentage'] = data['Claimant count'] / data['Population estimate'] * 100
data[data['Claimants percentage'].notna()].reset_index().geography_code.str.slice(0, 1).unique()

In [ ]:
DATA_FILE = '../data/processed/claimants/claimants-per-population-latest.csv'
os.makedirs(os.path.dirname(DATA_FILE), exist_ok=True)
data.to_csv(DATA_FILE)

In [ ]:
map_data = (
  gpd.read_file('../data/reference/pcon.geojson')
    .merge(
      data.reset_index().pivot(index='geography_code', columns='age', values='Claimants percentage'),
      left_on='PCON21CD',
      right_index=True,
      how='outer'
    )
)

In [ ]:
map_data.columns

In [ ]:
import matplotlib.pyplot as plt

map_data.plot(
    column='Aged 16-24',
    legend=True,
    legend_kwds={"label": "% Claimants per capita (Aged 16-24)", "orientation": "vertical"},
    figsize=(10,10)
).set_axis_off()